In [1]:
import mysql.connector
import uuid
import random
import datetime
import hashlib
from faker import Faker

# Initialize Faker for generating descriptions
fake = Faker()

# Database connection configuration
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',  # Replace with your MySQL username
    'password': 'rootpassword',  # Replace with your MySQL password
    'database': 'lbdr_db'
}

# Configuration
SQL_FILE_PATH = './Database/test_data.sql'
DB_NAME = 'lbdr_db'

# Data pools
first_names = ["Alice", "Bob", "Charlie", "David", "Emma", "Fiona", "George", "Hannah", "Ian", "Julia",
               "Kevin", "Laura", "Michael", "Nina", "Oliver", "Paul", "Quinn", "Rachel", "Steven", "Tina"]
last_names = ["Martin", "Dupont", "Schneider", "Rossi", "Smith", "Nguyen", "Kumar", "Lopez", "Brown", "Müller"]
sexes = ["M", "F", "Other"]
board_games = ["Fallout", "King of New York", "King of Tokyo", "Zombicide", 
               "Darkest Dungeon", "Root", "Pour la reine", "Gloomhaven", "Pandemic"]
jdr_games = ["1D8", "7ème Mer", "Advanced Bernard&Jean", "Adventure Party", "Agone", 
             "Alien", "An 1000", "Anima", "Appel de Cthulhu", "ARIA"]



In [2]:
import re

# Read plateau games file
with open('./jeux de plateau.txt', 'r', encoding='utf-8') as f:
    plateau_html = f.read()

# Read JDR games file
with open('./JDR.txt', 'r', encoding='utf-8') as f:
    jdr_html = f.read()

# Regex to extract option texts
board_games = re.findall(r'<option[^>]*>([^<]+)</option>', plateau_html)
jdr_games = re.findall(r'<option[^>]*>([^<]+)</option>', jdr_html)

# Filter out placeholders
board_games = [g.strip() for g in board_games if g.strip() and not g.strip().lower().startswith('jeu de plateau')]
# Remove duplicates
board_games = list(dict.fromkeys(board_games))

jdr_games = [g.strip() for g in jdr_games if g.strip() and not g.strip().startswith('--')]
# Remove duplicates and sort a bit
jdr_games = list(dict.fromkeys(jdr_games))

# Additional plateau games
additional_plateau = ["Darkest Dungeon", "Root", "Pour la reine", "Gloomhaven", "Pandemic"]
for g in additional_plateau:
    if g not in board_games:
        board_games.append(g)
        
board_games = list(set(board_games))
jdr_games = list(set(jdr_games))

In [3]:

def random_date(start_year=1970, end_year=2002):
    start = datetime.date(start_year, 1, 1)
    end = datetime.date(end_year, 12, 31)
    return start + datetime.timedelta(days=random.randint(0, (end - start).days))

def random_registration_date():
    start = datetime.date(2020, 1, 1)
    end = datetime.date.today()
    return start + datetime.timedelta(days=random.randint(0, (end - start).days))

def make_user(fn, ln, user_type, pwd_plain, used_usernames, used_emails):
    uid = str(uuid.uuid4())
    bd = random_date().isoformat()
    sex = random.choice(sexes)
    discord = f"{fn.lower()}#{random.randint(1000,9999)}"
    pseudo = fn.lower() + str(random.randint(1,9999))
    
    # Generate unique username
    attempt = 0
    while True:
        suffix = str(random.randint(1, 9999)) if attempt > 0 else str(random.randint(1,99))
        username = f"{fn.lower()}.{ln.lower()}{suffix}"
        email = f"{username}@example.com"
        if username not in used_usernames and email not in used_emails:
            used_usernames.add(username)
            used_emails.add(email)
            break
        attempt += 1
    
    salt = uuid.uuid4().hex[:8]
    pwd_hash = hashlib.sha256((salt + pwd_plain).encode()).hexdigest()
    reg_date = random_registration_date().isoformat() if user_type != "NON_REGISTERED" else None
    return {
        'id': uid,
        'first_name': fn,
        'last_name': ln,
        'birth_date': bd,
        'sex': sex,
        'discord_id': discord,
        'pseudonym': pseudo,
        'email': email,
        'username': username,
        'password_hash': pwd_hash,
        'password_salt': salt,
        'user_type': user_type,
        'registration_date': reg_date,
        'old_user': False,
        'first_connection': False
    }

def generate_description(game_name):
    return fake.paragraph(nb_sentences=3, variable_nb_sentences=True).replace("'", "''")

def generate_short_description(game_name, game_type):
    return f"{game_type} de {game_name}".replace("'", "''")

def get_first_sunday(month, year=2025):
    d = datetime.date(year, month, 1)
    while d.weekday() != 6:  # Sunday
        d += datetime.timedelta(days=1)
    return d


In [4]:

# Connect to database
try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("Connected to database")
except mysql.connector.Error as err:
    print(f"Error connecting to database: {err}")
    exit(1)


Connected to database


In [5]:
# Generate users
users = []
used_usernames = set()
used_emails = set()
for i in range(50):
    fn = first_names[i % len(first_names)]
    ln = last_names[i % len(last_names)]
    users.append(make_user(fn, ln, "REGISTERED", fn.lower(), used_usernames, used_emails))
users.append(make_user("Admin", "User", "ADMINISTRATOR", "admin", used_usernames, used_emails))

# Insert users
user_insert = """
INSERT INTO users (id, first_name, last_name, birth_date, sex, discord_id, pseudonym, email, username, 
                 password_hash, password_salt, user_type, registration_date, old_user, first_connection)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
for user in users:
    cursor.execute(user_insert, (
        user['id'], user['first_name'], user['last_name'], user['birth_date'], user['sex'],
        user['discord_id'], user['pseudonym'], user['email'], user['username'],
        user['password_hash'], user['password_salt'], user['user_type'],
        user['registration_date'], user['old_user'], user['first_connection']
    ))
    print(user)

{'id': 'f6b2bfe8-41a0-43d4-92e6-761f671b0fe8', 'first_name': 'Alice', 'last_name': 'Martin', 'birth_date': '1976-07-27', 'sex': 'F', 'discord_id': 'alice#6044', 'pseudonym': 'alice9036', 'email': 'alice.martin45@example.com', 'username': 'alice.martin45', 'password_hash': 'a368e29d6fe80901ffe4e3c0be835a830398fe1eef8b7f7432bff2af16e18a10', 'password_salt': '8662d046', 'user_type': 'REGISTERED', 'registration_date': '2020-06-29', 'old_user': False, 'first_connection': False}
{'id': 'e7a1efbc-ba79-4355-8010-96be491c8fe6', 'first_name': 'Bob', 'last_name': 'Dupont', 'birth_date': '2002-01-09', 'sex': 'F', 'discord_id': 'bob#1942', 'pseudonym': 'bob3660', 'email': 'bob.dupont72@example.com', 'username': 'bob.dupont72', 'password_hash': 'a25067eefe519c4403bbd97c8d22b2e37af58f4f47ea0f80719bf1f7a5573722', 'password_salt': '6384a39b', 'user_type': 'REGISTERED', 'registration_date': '2020-11-03', 'old_user': False, 'first_connection': False}
{'id': '80fb82e6-1851-4500-a59a-0713592a0138', 'first_

In [6]:

# Insert locations
locations = [
    {"name": "Saint Vincent Center", "address": "123 Rue Saint Vincent, Orléans", "latitude": 47.9029, "longitude": 1.9040},
    {"name": "Orléans Community Center", "address": "456 Avenue de la Communauté, Orléans", "latitude": 47.9129, "longitude": 1.9140}
]
location_insert = """
INSERT INTO locations (name, address, latitude, longitude, description)
VALUES (%s, %s, %s, %s, %s)
"""
location_ids = {}
for loc in locations:
    cursor.execute(location_insert, (
        loc['name'], loc['address'], loc['latitude'], loc['longitude'],
        f"Description for {loc['name']}".replace("'", "''")
    ))
    cursor.execute("SELECT LAST_INSERT_ID()")
    location_ids[loc['name']] = cursor.fetchone()[0]


In [7]:

# Insert regular schedules
schedule_insert = """
INSERT INTO location_schedule (location_id, start_time, end_time, recurrence_type, recurrence_pattern)
VALUES (%s, %s, %s, %s, %s)
"""
cursor.execute(schedule_insert, (
    location_ids["Saint Vincent Center"], "19:15:00", "01:00:00", "WEEKLY",
    '{"byDay":"FR","interval":1}'
))
cursor.execute(schedule_insert, (
    location_ids["Orléans Community Center"], "14:00:00", "20:00:00", "WEEKLY",
    '{"byDay":"SA","interval":1}'
))


In [8]:

# Insert association weekend events and schedules
event_insert = """
INSERT INTO events (name, description, start_date, end_date, location_id)
VALUES (%s, %s, %s, %s, %s)
"""
event_ids = []
for month in range(1, 7):
    sunday = get_first_sunday(month)
    friday = sunday - datetime.timedelta(days=2)
    cursor.execute(event_insert, (
        f"Association Weekend {month}", "Monthly gaming event", friday.isoformat(),
        sunday.isoformat(), location_ids["Saint Vincent Center"]
    ))
    cursor.execute("SELECT LAST_INSERT_ID()")
    event_id = cursor.fetchone()[0]
    event_ids.append(event_id)
    
    # Friday schedule
    cursor.execute(schedule_insert, (
        location_ids["Saint Vincent Center"], "19:15:00", "01:00:00", "NONE",
        f'{{"event_id":{event_id}}}'
    ))
    # Saturday schedule
    cursor.execute(schedule_insert, (
        location_ids["Saint Vincent Center"], "10:00:00", "01:00:00", "NONE",
        f'{{"event_id":{event_id}}}'
    ))
    # Sunday schedule
    cursor.execute(schedule_insert, (
        location_ids["Saint Vincent Center"], "10:00:00", "01:00:00", "NONE",
        f'{{"event_id":{event_id}}}'
    ))


In [9]:

# Insert games
game_insert = """
INSERT INTO games (name, description, type)
VALUES (%s, %s, %s)
"""
game_ids = {}
all_games = board_games + jdr_games
for game in all_games:
    game_type = "BOARD_GAME" if game in board_games else "JDR"
    try:
        cursor.execute(game_insert, (
            game, generate_description(game), game_type
        ))
        cursor.execute("SELECT LAST_INSERT_ID()")
        game_ids[game] = cursor.fetchone()[0]
    
    except	: 
        print('This ' + game +' game is in double')

# Generate parties
parties = []
closed_campaigns = 25
open_campaigns = 8
oneshots = 7  # 7 regular + 8 board game one-shots = 15 total one-shots
board_game_oneshots = 8
total_parties = closed_campaigns + open_campaigns + oneshots + board_game_oneshots

# Closed campaigns
for _ in range(closed_campaigns):
    game = random.choice(all_games)
    max_players = random.randint(4, 7)
    parties.append({
        "type": "CAMPAIGN", "campaign_type": "CLOSED", "game": game,
        "max_players": max_players, "mj_id": random.choice(users)['id']
    })

# Open campaigns
for _ in range(open_campaigns):
    game = random.choice(all_games)
    max_players = random.randint(4, 7)
    parties.append({
        "type": "CAMPAIGN", "campaign_type": "OPEN", "game": game,
        "max_players": max_players, "mj_id": random.choice(users)['id']
    })

# Regular one-shots
for _ in range(oneshots):
    game = random.choice(all_games)
    max_players = random.randint(4, 7)
    parties.append({
        "type": "ONESHOT", "campaign_type": None, "game": game,
        "max_players": max_players, "mj_id": random.choice(users)['id']
    })

# Board game one-shots on Association Weekend Sundays
sunday_dates = [get_first_sunday(month) for month in range(1, 7)]
sunday_dates = sunday_dates[:4] * 2  # 8 dates (2 per month for first 4 months)
for date in sunday_dates:
    game = random.choice(board_games)
    max_players = random.randint(4, 7)
    parties.append({
        "type": "BOARD_GAME", "campaign_type": None, "game": game,
        "max_players": max_players, "mj_id": random.choice(users)['id'],
        "session_date": date
    })


This Fallout game is in double
This Root game is in double
This Le domaine des humains game is in double


In [10]:

# Insert parties
partie_insert = """
INSERT INTO partie (game_id, mj_id, partie_type, campaign_type, short_description, description, max_players)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
partie_ids = []
for p in parties:
    game_type = "BOARD_GAME" if p['game'] in board_games else "JDR"
    cursor.execute(partie_insert, (
        game_ids[p['game']], p['mj_id'], p['type'], p['campaign_type'],
        generate_short_description(p['game'], p['type']), generate_description(p['game']),
        p['max_players']
    ))
    cursor.execute("SELECT LAST_INSERT_ID()")
    partie_ids.append(cursor.fetchone()[0])


In [11]:

# Insert partie_members
partie_members_insert = """
INSERT INTO partie_members (partie_id, user_id)
VALUES (%s, %s)
"""
partie_members = []
for idx, p in enumerate(parties):
    if p['type'] == "CAMPAIGN":
        maxp = p['max_players']
        if p['campaign_type'] == "OPEN":
            count = maxp + random.randint(1, 3)
        else:
            count = random.randint(1, maxp)
        selected_users = random.sample(users, min(count, len(users)))
        for user in selected_users:
            cursor.execute(partie_members_insert, (partie_ids[idx], user['id']))
            partie_members.append((partie_ids[idx], user['id']))


In [12]:

# Generate sessions (Jan 1, 2025 to Jun 30, 2025)
session_insert = """
INSERT INTO sessions (partie_id, location_id, session_date, start_time, end_time, mj_id)
VALUES (%s, %s, %s, %s, %s, %s)
"""
sessions = []
for idx, p in enumerate(parties):
    if p['type'] == "CAMPAIGN":
        # Random date in first half of 2025
        session_date = datetime.date(2025, random.randint(1, 6), random.randint(1, 28))
        start_time = "19:15:00"
        end_time = "22:15:00"
        mj_id = p['mj_id']
        cursor.execute(session_insert, (
            partie_ids[idx], location_ids["Saint Vincent Center"],
            session_date, start_time, end_time, mj_id
        ))
        cursor.execute("SELECT LAST_INSERT_ID()")
        sessions.append({"id": cursor.fetchone()[0], "partie_id": partie_ids[idx]})
    elif p['type'] == "BOARD_GAME" and 'session_date' in p:
        # Board game one-shots on Sundays
        start_time = "14:00:00"
        end_time = "17:00:00"
        mj_id = p['mj_id']
        cursor.execute(session_insert, (
            partie_ids[idx], location_ids["Saint Vincent Center"],
            p['session_date'], start_time, end_time, mj_id
        ))
        cursor.execute("SELECT LAST_INSERT_ID()")
        sessions.append({"id": cursor.fetchone()[0], "partie_id": partie_ids[idx]})
    elif p['type'] == "ONESHOT":
        # Regular one-shots on Fridays
        session_date = datetime.date(2025, random.randint(1, 6), random.randint(1, 28))
        while session_date.weekday() != 4:  # Friday
            session_date += datetime.timedelta(days=1)
        start_time = "19:15:00"
        end_time = "22:15:00"
        mj_id = p['mj_id']
        cursor.execute(session_insert, (
            partie_ids[idx], location_ids["Saint Vincent Center"],
            session_date, start_time, end_time, mj_id
        ))
        cursor.execute("SELECT LAST_INSERT_ID()")
        sessions.append({"id": cursor.fetchone()[0], "partie_id": partie_ids[idx]})


In [13]:

# Insert session_players
session_players_insert = """
INSERT INTO session_players (session_id, user_id)
VALUES (%s, %s)
"""
for session in sessions:
    partie_id = session['partie_id']
    partie_idx = partie_ids.index(partie_id)
    p = parties[partie_idx]
    maxp = p['max_players']
    if p['type'] == "CAMPAIGN":
        # Only partie_members can register
        members = [m[1] for m in partie_members if m[0] == partie_id]
        if members:
            selected = random.sample(members, min(len(members), maxp))
            for user_id in selected:
                cursor.execute(session_players_insert, (session['id'], user_id))
    else:
        # Any user can register for one-shots and board games
        selected = random.sample(users, min(len(users), maxp))
        for user in selected:
            cursor.execute(session_players_insert, (session['id'], user['id']))


In [14]:

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()
print("Test data generated and inserted into database")

Test data generated and inserted into database


In [ ]:
# Optionally, write SQL file for reference
with open(SQL_FILE_PATH, 'w', encoding='utf-8') as f:
    f.write(f"USE {DB_NAME};\n\n")
    # Users
    f.write("INSERT INTO users (id, first_name, last_name, birth_date, sex, discord_id, pseudonym, email, username, password_hash, password_salt, user_type, registration_date, old_user, first_connection) VALUES\n")
    f.write(",\n".join([f"('{u['id']}', '{u['first_name']}', '{u['last_name']}', '{u['birth_date']}', '{u['sex']}', '{u['discord_id']}', '{u['pseudonym']}', '{u['email']}', '{u['username']}', '{u['password_hash']}', '{u['password_salt']}', '{u['user_type']}', {'NULL' if u['registration_date'] is None else f'\'{u['registration_date']}\''}, {u['old_user']}, {u['first_connection']})" for u in users]) + ";\n\n")
    # Locations
    f.write("INSERT INTO locations (name, address, latitude, longitude, description) VALUES\n")
    f.write(",\n".join([f"('{loc['name']}', '{loc['address']}', {loc['latitude']}, {loc['longitude']}, 'Description for {loc['name']}')" for loc in locations]) + ";\n\n")
    # Games
    f.write("INSERT INTO games (name, description, type) VALUES\n")
    f.write(",\n".join([f"('{game}', '{generate_description(game)}', '{'BOARD_GAME' if game in board_games else 'JDR'}')" for game in all_games]) + ";\n\n")
    # Parties, partie_members, sessions, and session_players would require mapping IDs from DB
    f.write("-- Additional inserts for parties, members, sessions, and session_players are generated directly in the database\n")
print(f"SQL file written to {SQL_FILE_PATH}")